In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('../styles/custom.css', 'r').read()
    return HTML(styles)
css_styling()

# Vortex sheets
Our numerical method for 2D-potential flow will be developed using vortex sheets. In this first lesson, we derive the equations for a simple vortex sheet and plot the velocity fields.

## Potential Function
Consider a 2D vortex sheet defined along a curve $\cal S$. The sheet is characterized by a *strength* $\gamma(s)$, where $s$ is the coordinate along the sheet. An infinitesimal segment of this sheet is an infinitesimal point-vortex with strength $d\Gamma = \gamma ds$. Therefore we can integrate the potential for a point vortex

$$ \phi = \frac{\Gamma}{2\pi}\theta $$

to define the potential function for a vortex sheet as

$$ \phi(x,y) = \int_{\cal S} \frac{\gamma(s)}{2\pi}\theta(x,y,s)\ ds $$

where $\theta$ is the angle from $s$ to the point at which we evaluate the potential. 

---
![image](resources/graphics1.png)

---

For a vortex sheet defined from $-S,S$ along the $x$-axis, $\theta=\tan^{-1}(y/(x-s))$. If the strength is constant along the sheet, the potential is

$$ \phi(x,y) = \frac{\gamma}{2\pi}\int^S_{-S} \tan^{-1}\left(\frac y{x-s}\right)ds $$

## Velocity Field

The velocity is defined from the potential as

$$ u = \frac{\partial\phi}{\partial x}, \quad v = \frac{\partial\phi}{\partial y} $$

Therefore, the $u$ velocity is

$$u(x,y) = \frac{\gamma}{2\pi}\int^S_{-S} \frac{-y}{y^2+(x-s)^2} ds $$

substitute $t = (x-s)/y$, $dt = -ds/y$ and integrate to get

$$u(x,y) = \frac{\gamma}{2\pi}\left[\tan^{-1}\left(\frac{x-S}y\right)-\tan^{-1}\left(\frac{x+S}y\right)\right].$$

While the $v$ velocity is

$$v(x,y) = \frac{\gamma}{2\pi}\int^S_{-S} \frac{x-s}{y^2+(x-s)^2} ds $$

substitute $t = (x-s)^2+y^2$, $dt = -2(x-s)ds$ and integrate to get

$$v(x,y) =\frac{\gamma}{4\pi} \log\left(\frac{(x+S)^2+y^2}{(x-S)^2+y^2}\right).$$

That's it! For a given $\gamma$ and $S$ we can determine the velocity at any point $x,y$ by evaluating some $\tan^{-1}$ and $\log$ functions.

## Numerical implementation

To visualize the velocity field we will discretize the background space into a uniform grid and evaluate the functions above at each point. 

We need to `import numpy` to do this. This imports numerical functions like `linspace` to evenly divide up a line segment into an array points, and `meshgrid` which takes two one-dimensional arrays and creates two two-dimensional arrays to fill the space.

We also need to `import pyplot` which has a large set of plotting functions similar to matlab, such as a scatter plot.

In [ ]:
# only need to import and set parameters once in a notebook #
import numpy                             # import numerical library
from matplotlib import pyplot            # import plot library
%matplotlib inline

# set up grid
N = 30                          # number of points along each axis
X = numpy.linspace(-2, 2, N)    # computes a 1D-array for x
Y = numpy.linspace(-2, 2, N)    # computes a 1D-array for y
x, y = numpy.meshgrid(X, Y)     # generates a mesh grid

# plot it
pyplot.scatter(x, y)            # plot grid points 
pyplot.xlabel('x')              # label x 
pyplot.ylabel('y')              # label y

As expected, we get a grid of equally space points. You can change `N` in the code above and see how the resolution of this grid changes.

Next, we use the equations above to determine the velocity at each point in terms of the source strength and sheet extents. Since we'll use this repeatedly, we will write it as a set of functions.


##### Coding fundamental: Functions
##### Don't ever write the same code twice!

If you aren't familiar with defining functions in python, [read up](http://en.wikibooks.org/wiki/A_Beginner%27s_Python_Tutorial/Functions). 

So let's define function to evaluate the x-component and y-component derived above:

In [ ]:
def get_u( x, y, S, gamma ):
    return gamma/(2*numpy.pi)*(numpy.arctan((x-S)/y)-numpy.arctan((x+S)/y))

def get_v( x, y, S, gamma ):
    return gamma/(4*numpy.pi)*(numpy.log(((x+S)**2+y**2)/((x-S)**2+y**2)))

Not the prettiest equations, but nothing `numpy` can't handle. 

Now let's make a function to plot the flow. We'll use arrows to show the velocity vectors (`quiver`) and color contours to show the velocity magnitude (`contourf` with `colorbar` for the legend). 

In [ ]:
def plot_uv(u,v):
    pyplot.figure(figsize=(6,5))                    # set figure size
    m = numpy.sqrt(u**2+v**2)                       # compute velocity magnitude
    velocity = pyplot.contourf(x, y, m, vmin=0)     # plot magnitude contours
    pyplot.quiver(x, y, u, v)                       # plot vector field
    cbar = pyplot.colorbar(velocity)                # color bar
    cbar.set_label('Velocity magnitude');           # label color
    pyplot.xlabel('x')                              # label x
    pyplot.ylabel('y')                              # label y

Now we can compute the velocity on the grid and plot it

In [ ]:
# compute the velocity
gamma = -4               # sheet strength
S = 1                    # sheet extents
u = get_u(x,y,S,gamma)
v = get_v(x,y,S,gamma)

# plot it
plot_uv(u,v)
pyplot.plot([-min(S,2),min(S,2)],[0,0],'k-',lw=2) # draw the vortex sheet

##### Quiz 1

What shape do the streamlines make when you are sufficiently far from the body?

1. Ellipses
2. Circles
3. Straight lines 

(Hint: This is an *interactive* notebook - which parameter can you vary to answer the question?)

##### Quiz 2

What is the u,v velocity of points very near the center of the vortex sheet?

1. $u=0,\ v=\sqrt\gamma$
2. $u=\pm\frac 12 \gamma,\ v=0$
3. $u=\pm\gamma^2,\ v=0$

(Hint: $tan^{-1}(\pm \infty) = \pm \frac \pi 2$ )

## Background flow

Next, lets add a uniform background flow with magnitude one at angle $\alpha$.

$$ U_\infty = \cos\alpha,\quad V_\infty = \sin\alpha $$

Using *superposition* the total velocity is just the sum of the vortex sheet and uniform flow.

In [ ]:
alpha = numpy.pi/10      # free-stream angle
U_inf = numpy.cos(alpha) # free-stream in x
V_inf = numpy.sin(alpha) # free-stream in y

# superimpose to get velocity
gamma = -4               # sheet strength
S = 0.5                  # sheet extents
u = U_inf+get_u(x,y,S,gamma)
v = V_inf+get_v(x,y,S,gamma)

# plot it 
plot_uv(u,v)
pyplot.plot([-min(S,2),min(S,2)],[0,0],'k-',lw=2) # draw the vortex sheet

The dark blue circle is a stagnation point, ie the fluid has stopped, ie $u=v=0$.

##### Quiz 3

How can you make the stagnation point touch the vortex sheet?

1. Set $\alpha=0$
2. Set $\gamma=\pm 2 U_\infty$
3. Set $S = \infty$

## General vortex panel

Finally, we would like to be able to compute the flow induced by a flat vortex sheet defined between any two points $x_0,y_0$ and $x_1,y_1$. A sheet so defined is called a *vortex panel*.  We could start from scratch, re-deriving the potential and velocity fields. But why would we want to do that?

##### Coding fundamental: Reuse
##### Recast problems to reuse existing code!

By switching from the global coordinates to a *panel-based* coordinate system, we can transform any vortex panel to match our previous example. After computing the velocity using our old functions, we just need to rotate the vector back to the global coordinates.

---
![image](resources/graphics0.png)

---

## VortexPanel module

I've written a set of functions which follows the method described above to define a general vortex panel and compute the velocity which it induces. These functions define a `Panel`, and are stored in the file `VortexPanel.py` in the same folder as this notebook.

Just as we used `from matplotlib import pyplot` above, we can import `Panel` from the `VortexPanel` module:

In [ ]:
from VortexPanel import Panel

This makes the `Panel` class available. You can use the `help` function to get a summary of the class without checking the code.

In [ ]:
help(Panel)

##### Quiz 4

What is the function used to draw a panel named `p`?

1. `p.plot()`
1. `p.show()`
1. `p.display()`

Now we can define a general panel and compute its velocity using the `Panel.velocity()` function. 

In [ ]:
# define panel
my_panel = Panel(x0=-0.7,y0=0.5,x1=0.5,y1=-0.4,gamma=-2)

# compute velocity on grid
u,v = my_panel.velocity(x,y)

# plot it
plot_uv(u,v)      # plot the flow on the grid
my_panel.plot()   # plot the panel

##### Your turn #1

Use superposition to compute a flow moving between **two** parallel plates with opposite vortex strength.

 - ** Complete ** the code below by adding the contribution of the panels.
 - ** Adjust ** the strength until the flow stagnates.
 - ** Discuss ** what types of flow you could model with more panels

---

##### Solution #1

In [ ]:
# Your turn!
gamma = 1               # strength
yc = 0.5                # spacing

panel_1 = Panel(x0=-1,y0= yc,x1=1,y1= yc,gamma= gamma)
panel_2 = Panel(x0=-1,y0=-yc,x1=1,y1=-yc,gamma=-gamma)

u = numpy.ones_like(x)          # free-stream in x
v = numpy.zeros_like(y)         # free-stream in y

# compute velocity by superposition
### your code here

# plot it
plot_uv(u, v)           # plot the flow on the grid
panel_1.plot()          # plot panel_1
panel_2.plot()          # plot panel_2